In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "wiki-research-l/2020-July"
file_list = getFileList(directory_path)
file_list.sort()

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [3]:
bodies = dict(Bodies=[])
for idx, body in mail_df.loc[:,'Body'].items():
    origin = []
    greetings = dict(Greetings=[])
    sentences = dict(Sentence=[])
    captions = dict(Caption=[])
    bulletlist = dict(Bulletlist=[])
    ending = dict(Ending=[])
    quotation = dict(Quotation=[])
    footer = dict(Footer=[])
    misc = dict(Misc=[])

    lines = body.splitlines()
    for num, line in enumerate(lines):
        if re.match(r'\[G\]',line) is not None:
            greetings['Greetings'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[S\]',line) is not None:
            sentences['Sentence'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[C\]',line) is not None:
            captions['Caption'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[B\]',line) is not None:
            bulletlist['Bulletlist'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[E\]',line) is not None:
            ending['Ending'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[Q\]',line) is not None:
            quotation['Quotation'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[F\]',line) is not None:
            footer['Footer'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[M\]',line) is not None:
            misc['Misc'].append({num:line[3:]})
            origin.append(line[3:])
        else: #空白行に対応する
            continue
    originbody = ' '.join(origin)
    bodies['Bodies'].append({'idx':idx, 'countrows':len(lines), 'body':originbody, **greetings, **sentences, **captions, **bulletlist, **ending, **quotation, **footer, **misc})

In [4]:
#　bodiesのsentenceのvalueから複数文のテキストを作る
s_list = []
for i in bodies['Bodies']:  #1通ずつ取り出す
    text = ''
    for j in i['Sentence']: #1行{行番号:文}ずつ取り出し，複数文が含まれた1つの文字列にする
        text = text + list(j.values())[0] + ' '
    s_list.append(text) #1通ごとの自然文のテキストをリストに格納する

In [14]:
# センテンスのdataframeを作る
sentence_list = []
# 文章全体に対する前処理
for i, content in enumerate(s_list):
    sentences = tokenize.sent_tokenize(content)
    # 文に対する前処理
    for j, sentence in enumerate(sentences):
        sentence = re.sub(r'\s{2,}',' ',sentence)
        sentence_list.append([mail_df['Message-ID'][i],sentence])
sentence_df = pd.DataFrame(sentence_list,columns=['Message-ID','sentence'])

In [55]:
sentence_df.head()

,Message-ID,sentence
0,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,Thanks for your reply!
1,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,It's a great suggestion to include a list of ORES based tools for people who are not familiar with ORES itself.
2,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,"For our research, we would love to interview you and test our visualizations."
3,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,Please let me know what times will work best for you so we can schedule a meeting.
4,<CAE4fJj_b7k8yeqmz19a-seo3gtL9Mg1nvQO0oPUtrxvCFMkEaw@mail.gmail.com>,Thanks for clarifying.


In [7]:
'''
# OpenIEにかける
from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for sentence in sentence_list:
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

# OIEの抽出結果を整形する
rows = []
for extractions in extractions_list:
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [sentence, arg1, rel, arg2s, confidence]
            rows.append(row)

# 整形結果をdataframeにする
kb_df = pd.DataFrame(rows, columns = ['sentence', 'arg1', 'rel', 'arg2s', 'confidence'])

# dataframeをcsv出力する
kb_df.to_csv('wiki-research-l/output/triple_from_text_part.csv')

# dataframeをRDBのテーブルにする
'''
'''
from db import connect
engine = connect()
kb_df.to_sql(name='kb_wiki_research_l_text',con=engine,if_exists='replace',index=None)
'''


In [44]:
from entityLinking import tagme, confidentAnnotations, mediaWiki

entity_list = []
for i, sentence in sentence_df['sentence'].items():
    json_res = tagme(sentence)
    for candidate in json_res['annotations']:
        if candidate['rho'] > 0.5:
            entity_list.append([i, candidate['spot'],candidate['rho'],candidate['id'],candidate['title']])
        else:
            continue

In [53]:
rows = []
for row in entity_list:
    rows.append([sentence_df['Message-ID'][row[0]], row[1], row[2], row[3], row[4]])
entity_df = pd.DataFrame(rows,columns=['Message-ID','spot','rho','id','title'])

In [54]:
entity_df.head()

,Message-ID,spot,rho,id,title
0,<CALeA2c9GW9BFSMetuSCDWLbm2iGG1xM7Bp0R2A9-JJF2_7Xrwg@mail.gmail.com>,https,0.507675,13443,Hypertext Transfer Protocol
1,<CAD_=H2LWP6n5COjqc-DEw59feJGxXBpJLen4m7BtPT5MfcJrpg@mail.gmail.com>,Wikimedia Foundation,0.594803,18618509,Wikimedia Foundation
2,<CAD_=H2LWP6n5COjqc-DEw59feJGxXBpJLen4m7BtPT5MfcJrpg@mail.gmail.com>,University of Turin,0.634261,453828,University of Turin
3,<CAD_=H2LWP6n5COjqc-DEw59feJGxXBpJLen4m7BtPT5MfcJrpg@mail.gmail.com>,Phabricator,0.567908,40879657,Phabricator
4,<CAD_=H2LWP6n5COjqc-DEw59feJGxXBpJLen4m7BtPT5MfcJrpg@mail.gmail.com>,Wikimedia Foundation,0.601363,18618509,Wikimedia Foundation
